In [1]:
# ! pip install PyPrind

In [4]:
import pyprind
import pandas as pd
import os
# change the `basepath` to the directory of the
# unzipped movie dataset
basepath = 'aclImdb'

In [ ]:
labels = {'pos': 1, 'neg': 0}

In [ ]:
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()

In [ ]:
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file),'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]],ignore_index=True)
            pbar.update()

In [ ]:
df.columns = ['review', 'sentiment']

In [ ]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [5]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


## Introducing the bag-of-words model

In [10]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining','The weather is sweet','The sun is shining and the weather is sweet'])
bag = count.fit_transform(docs)

In [11]:
# it is only creating a dictionary and in this word is mapped to an index which means in dictionary
# sequence is: [and,is,shining,sun,sweet,the,weather]
print(count.vocabulary_)

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}


In [12]:
print(bag.toarray()) # it creates a feature vector for the dictionary and values here represents a count.
# 0111010 means in first document, "and" comes 0 times, "is" comes 1 times, "shining" comes 1 time and so on 

[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]


In [13]:
# These values
# in the feature vectors are also called the raw term frequencies: tf (t,d )—the number
# of times a term t occurs in a document d.

## Assessing word relevancy via term frequency-inverse document frequency

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,norm='l2',smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.56 0.56 0.   0.43 0.  ]
 [0.   0.43 0.   0.   0.56 0.43 0.56]
 [0.4  0.48 0.31 0.31 0.31 0.48 0.31]]


## Cleaning text data

In [15]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [16]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text = (re.sub('[\W]+', ' ', text.lower()) +' '.join(emoticons).replace('-', ''))
    return text

In [17]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [18]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [19]:
df['review'] = df['review'].apply(preprocessor)

## Processing documents into tokens

In [20]:
def tokenizer(text):
    return text.split()

In [21]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [22]:
# ! pip install nltk

In [23]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ankit19.gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

## Training a logistic regression model for document classification

In [26]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)
param_grid = [{'vect__ngram_range': [(1,1)],'vect__stop_words': [stop, None],'vect__tokenizer': [tokenizer,tokenizer_porter],'clf__penalty': ['l1', 'l2'],'clf__C': [1.0, 10.0, 100.0]},{'vect__ngram_range': [(1,1)],'vect__stop_words': [stop, None],'vect__tokenizer': [tokenizer,tokenizer_porter],'vect__use_idf':[False],'vect__norm':[None],'clf__penalty': ['l1', 'l2'],'clf__C': [1.0, 10.0, 100.0]}]
lr_tfidf = Pipeline([('vect', tfidf),('clf',LogisticRegression(random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,scoring='accuracy',cv=5, verbose=1,n_jobs=1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\

C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\

C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\

C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\

...

C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale th

C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\

C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\

C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\

C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Self_Projects\Python_Machine_Learning_Sebastian_Raschka\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale th

In [ ]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

In [ ]:
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

In [ ]:
clf = gs_lr_tfidf.best_estimator_

In [ ]:
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

## Working with Bigger data - online algorithms and out-of-core learning

In [6]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
    + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [7]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [8]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [9]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [11]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',n_features=2**21,preprocessor=None,tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [12]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:19


In [13]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


In [14]:
clf = clf.partial_fit(X_test, y_test)

## Topic modeling with Latent Dirichlet Allocation

### Decomposing text documents with LDA

In [15]:
import pandas as pd
df = pd.read_csv('movie_data.csv', encoding='utf-8')

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english',max_df=.1,max_features=5000)
X = count.fit_transform(df['review'].values)

In [20]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10,random_state=123,learning_method='batch')
X_topics = lda.fit_transform(X)

In [21]:
lda.components_.shape

(10, 5000)

In [22]:
n_top_words = 5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i] for i in topic.argsort() [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


In [23]:
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorror movie #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


Horror movie #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

Horror movie #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

Horror movie #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...
